In [108]:
#imports
import numpy as np
import astropy 
from astropy import table
from astropy.io import ascii
from astropy.table import Table, Column, join, vstack
import matplotlib.pyplot as plt
import matplotlib
from astropy import units as u
from astropy.coordinates import SkyCoord
%matplotlib inline

In [208]:
#read in files
Fang = ascii.read('../../OrionCatalogs/orionCatalogFang.tsv')
Megeath = ascii.read('../../OrionCatalogs/orionCatalogMegeath.tsv')
Esplin = ascii.read('../../WISEandGALEXmatchCatalogs/WISEcatalogWith2MASS.txt')
Koenig = ascii.read('../../KoenigSelectedCatalogs/Taurus/finalTaurusSelection_withWISE.txt')
Extinct1 = ascii.read('../../ExtinctionTargets/YSO_Orion_B_F1_AncillaryTargets_ARS.txt')
#col 3: RA  col 4: Dec

In [209]:
#read all files into a table, adding a region flag
#create new Table
Extinct=Table()  
for name in Extinct1.colnames:
    col = Column(name = name, dtype = Extinct1[name].dtype)
    Extinct.add_column(col)

#add region column
col = Column(name = 'region', dtype = 'S12', length = len(Extinct))
Extinct.add_column(col)
    
#define which files to read
filenames = ['../../ExtinctionTargets/YSO_TAU_L1495_AncillaryTargets_ARS.txt', 
            '../../ExtinctionTargets/YSO_TAU_L1521_AncillaryTargets_ARS.txt', 
            '../../ExtinctionTargets/YSO_TAU_L1527_AncillaryTargets_ARS.txt', 
            '../../ExtinctionTargets/YSO_TAU_L1536_AncillaryTargets_ARS.txt'] 
#define flag for each file
regions = ['L1495' ,'L1521', 'L1527', 'L1536']

#run through the list of files, adding each row in the file to a new table
for i in range(len(filenames)):
    test = open(filenames[i], 'r')
    for line in test:
        lineSplit = line.split()
        if len(lineSplit) > 3 and len(lineSplit) < 16:
            lineSplit.insert(13,regions[i])
            Extinct.add_row(lineSplit[:14])
    

In [210]:
#find known YSOs with H < 12.5 in each catalog
FangHmag = []
for i in range(len(Fang[2:])):
    if float(Fang['Hmag'][2:][i]) < 12.5:
        FangHmag.append(i)
        
MegeathHmag = []
for i in range(len(Megeath[2:])):
    if float(Megeath['Hmag'][2:][i]) < 12.5:
        MegeathHmag.append(i)
        
KoenigHmag = []
for i in range(len(Koenig)):
    if float(Koenig['h_m_2mass'][i]) < 12.5:
        KoenigHmag.append(i)

EsplinHmag = []
for i in range(len(Esplin)):
    if float(Esplin['h_m_2mass'][i]) < 12.5:
        EsplinHmag.append(i)


#find the separations between H < 12.5 YSOs and extinction targets
#restrict to within 70 arcseconds
from matchCatalog import matchCatalog
FangCoords = SkyCoord(Fang['_RAJ2000'][2:][FangHmag], Fang['_DEJ2000'][2:][FangHmag], unit = 'deg')
MegeathCoords = SkyCoord(Megeath['_RAJ2000'][2:][MegeathHmag], Megeath['_DEJ2000'][2:][MegeathHmag], unit = 'deg')
KoenigCoords = SkyCoord(Koenig['ra'][KoenigHmag], Koenig['dec'][KoenigHmag], unit = 'deg')

Esplin['ra'].unit = 'deg'
Esplin['dec'].unit = 'deg'
EsplinCoords = SkyCoord(Esplin['ra'][EsplinHmag], Esplin['dec'][EsplinHmag], unit = 'deg')

ExtinctCoords = SkyCoord(Extinct['col3'], Extinct['col4'], unit = 'deg')

#compare to Fang
#Fangmatches, fdist = matchCatalog(FangCoords, ExtinctCoords, 70.0, False)

#compare to Megeath
#Megeathmatches, mdist = matchCatalog(MegeathCoords, ExtinctCoords, 70.0, False)

#compare to Koenig
Koenigmatches, kdist = matchCatalog(KoenigCoords, ExtinctCoords, 70.0, False)

#compare to Esplin
Esplinmatches, edist = matchCatalog(EsplinCoords, ExtinctCoords, 70.0, False)

In [213]:
#write these sources to a table, combine tables and write to file.

#TooClosef = Table(Extinct[Fangmatches])
#col = Column(data = fdist, name = 'separation', dtype = 'float', length = len(TooClosef))
#TooClosef.add_column(col)

#TooClosef = table.unique(TooClosef, keys='col2')


#TooClosem = Table(Extinct[Megeathmatches])
#col = Column(data = mdist, name = 'separation', dtype = 'float', length = len(TooClosem))
#TooClosem.add_column(col)

#TooClosem = table.unique(TooClosem, keys='col2')

TooClosek = Table(Extinct[Koenigmatches])
col = Column(data = kdist, name = 'separation', dtype = 'float', length = len(TooClosek))
TooClosek.add_column(col)

TooClosek = table.unique(TooClosek, keys='col2')

TooClosee = Table(Extinct[Esplinmatches])
col = Column(data = edist, name = 'separation', dtype = 'float', length = len(TooClosee))
TooClosee.add_column(col)

TooClosee = table.unique(TooClosee, keys = 'col2')

#TooClose = join(TooClosef, TooClosem, join_type='outer')
#Final = join(TooClose, TooClosek, join_type='outer')
Final = join(TooClosee, TooClosek, join_type='outer')

Final.rename_column('col1', 'Priority')
Final.rename_column('col2', 'ID')
Final.rename_column('col3', 'RA(J2000)')
Final.rename_column('col4', 'Dec(J2000)')
Final.rename_column('col5', 'Epoch')
Final.rename_column('col6', 'Hmag')
Final.rename_column('col7', 'srcH')
Final.rename_column('col8', 'pmRA')
Final.rename_column('col9', 'pmDec')
Final.rename_column('col10', 'pmCatalog')
Final.rename_column('col11', 'type')


ascii.write(Final, '../../ExtinctionTargets/ConflictsTaurus.txt', exclude_names = ('col12', 'col13'))

In [212]:
Final

Priority,ID,RA(J2000),Dec(J2000),Epoch,Hmag,srcH,pmRA,pmDec,pmCatalog,type,col12,col13,region,separation
int64,string136,float64,float64,float64,float64,string40,string8,string8,string8,string32,string80,string48,string96,float64
1,J04391820+2544326,69.825863,25.742405,2000.0,10.17,2MASS,?,?,?,star,extinction,target,L1527,46.9239454312
29,J04162490+2808205,64.103755,28.13904,2000.0,10.91,2MASS,?,?,?,star,extinction,target,L1495,61.9433979846
40,J04191625+2829331,64.817709,28.492533,2000.0,10.71,2MASS,?,?,?,star,extinction,target,L1495,45.3083371193
